In [ ]:
%run ./Muehle_Utilities.ipynb

## Später noch entfernen: Testboards

In [ ]:
#Startboard
board1 = [[9, 9],
         [[0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0]]]
#Phase 1
board2 = [[7, 6],
         [[0, 0, 0, 0, 2, 0, 0, 0],
          [0, 1, 2, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 1, 2, 0, 0]]]
#Phase 2, keine Mühle
board3 = [[0, 0],
         [[1, 0, 1, 2, 0, 1, 0, 0],
          [2, 0, 1, 0, 0, 0, 1, 0],
          [0, 2, 0, 0, 1, 2, 0, 0]]]
#Phase 2, Mühle
board4 = [[0, 0],
         [[1, 1, 1, 2, 0, 1, 0, 0],
          [2, 0, 1, 0, 0, 0, 1, 0],
          [0, 2, 0, 0, 1, 2, 0, 0]]]
#Phase 2, Spieler 2 kann nicht mehr ziehen
board5 = [[0, 0],
         [[2, 1, 2, 1, 0, 0, 0, 1],
          [2, 2, 1, 0, 0, 1, 2, 1],
          [0, 1, 0, 0, 0, 0, 0, 0]]]
#Phase 3 für Spieler 1
board6 = [[0, 0],
         [[0, 0, 1, 2, 0, 1, 0, 0],
          [2, 0, 0, 0, 0, 0, 0, 0],
          [0, 2, 0, 0, 1, 2, 0, 0]]]
#Spieler 1 hat weniger als 3 Steine
board7 = [[0, 0],
         [[0, 0, 1, 2, 0, 1, 0, 0],
          [2, 0, 0, 0, 0, 0, 0, 0],
          [0, 2, 0, 0, 0, 2, 0, 0]]]

## Spielphasen

Die Funktion `get_player_phase()` prüft in welcher Phase sich der übergebenen Spieler befindet. Hat er noch setzbare Steine, ist also sein Wert in "remaining" größer null, befindert er sich in Phase eins. Ist dies nicht der Fall und auch auf dem Feld befinden sich nur noch drei Steine, so befindet der Spieler sich in der Springphase. Ist dem nicht so, so ist der Spieler in Phase zwei, der Zugphase.

In [ ]:
def get_player_phase(state, player):
    [remaining, board] = state
    if remaining[player-1] >= 1: # Es wurden noch nicht alle 9 Steine gesetzt
            return 1
    elif count_player_pieces(state, player) == 3: # nur noch drei Steine auf dem Spielfeld
        return 3
    else: # sonst
        return 2

`possible_positions_phase_two()` berechnet die nächsten möglichen klickbaren Positionen, falls der Spieler in Phase 2 (Zugphase) ist. Dies sind ausgehend vom gewählten Stein alle benachbarten leeren Felder.

In [ ]:
def possible_positions_phase_two(state, ring, cell):
    collected_piece = (ring, cell)
    neighboring = set(neighboring_positions(collected_piece))
    empty = set(empty_positions(state))
    possible_positions = neighboring.intersection(empty)
    return list(possible_positions)

`next_positions()` berechnet in welcher Phase sich der Spieler befindet und dementsprechend, wo er als nächstes klicken kann. Falls der Spieler in Phase 1 (Setzphase) oder Phase 3 (Springphase) ist, dies sind unabhängig vom aktuellen Spieler oder in Phase 3 auch unabhängig vom ausgewählten Stein alle leeren Felder. Für Phase 2 wird die zuvor definierte Funktion `possible_positions_phase_two()` aufgerufen.

In [ ]:
def next_positions(state, player, ring, cell):
    phase = get_player_phase(state, player)
    if phase == 1:
        return empty_positions(state)
    elif phase == 2:
        return possible_positions_phase_two(state, ring, cell)
    else:
        return empty_positions(state)

Die Funktion `finish()` prüft, ob das Spiel zu Ende ist und gibt ggf. den Gewinner zurück. Die Rückgabewerte sind 
* 0: das Spiel ist noch nicht zu Ende
* 1: Spieler 1 hat gewonnen
* 2: Spieler 2 hat gewonnen

Solange ein Spieler noch Steine setzen kann, ist das Spiel noch nicht entschieden. Ansonsten gibt es zwei Möglichkeiten, bei denen das Spiel noch nicht zu Ende ist. Die erste Situation tritt ein, wenn ein Spieler weniger als drei Steine hat. Er hat dann verloren. Bei der zweiten Option verliert ein Spieler, wenn er sich in Phase 2 befindet, aber nicht mehr ziehen kann. Dies kommt vor, wenn alle benachbarten Felder seiner Steine vom Gegenspieler besetzt sind.

In [ ]:
def finish(state):
    [remaining, board] = state
    if any(p > 0 for p in remaining): # Wenn man noch Steine setzen kann, hat man nicht verloren
        return 0
    number_pieces = [count_player_pieces(state, 1), count_player_pieces(state, 2)]
    for player in [1, 2]:
        # Wenn man weniger als 3 Steine hat, hat man verloren
        if number_pieces[player-1] < 3 :
            return opponent(player)
        else:
            # Wenn man in Phase 2 ist und nicht mehr ziehen kann, aber in Phase 3 ist, hat man verloren
            player_pieces = player_pieces(state, player)
            phase = get_player_phase(state, player)
            if not any(is_movable(state, position) for position in player_pieces) and phase == 2:
                return opponent(player)
    # Falls die beiden Endsituationen für keinen Spieler eingetreten sind, ist das Spiel nicht zu Ende
    return 0

## Spielablauf

Die Funktion `make_move()` ist eine komplexere Funktion und steuert den Spielablauf, bzw. das Setzen, Ziehen, Springen je nach Spielphase sowie das Entfernen von Steinen, falls Mühlen gebildet wurden.
Zunächst wird die Phase und demensprechend alle möglichen nächsten Positionen für einen Stein des aktuellen Spielers ermittelt. Müssen auf Grund von zuvor eventuell gebildeter Mühlen noch Steine des Gegners entfernt werden, passiert dies zuerst. Dazu wird überprüft, ob der angeklickte Stein ein schlagbarer Stein des Gegner ist. Ist dies der Fall, wird der Stein enfernt. Andernfalls wird eine Fehlermeldung geworfen und der Spieler kann einen anderen Stein zum Entfernen auswählen. Falls noch mehr Steine entfernt werden müssen, werden die möglichen Positionen markiert.

Dürfen keine Steine entfernt werden, kommt die nächste Aktion auf die Phase des Spielers an. In Phase 1 wird überprüft, ob das angeklickte Feld noch frei ist. Wenn ja, wird dort ein Stein des Spielers positioniert. Wenn nicht, wird eine Fehlermeldung geworfen und der Spieler erhält einen neuen Versuch. In Phase 2 oder 3 wird zunächst überprüft, ob der ausgewählte Stein ein eigener Stein ist. Wenn ja, werden alle möglichen Zielfelder markiert und die Variable `move_or_jump` auf True gesetzt. Beim nächsten Aufruf der Funktion wird geprüft, ob das jetzt angeklickte Feld tatsächlich ein Zielfeld des zuvor ausgewählten Steins ist. Ist dem so, wird der Stein dorthin verschoben, in dem das Zielfeld als eigenes und das Alte als leeres Feld markiert wird.

Schließlich wird überprüft, ob neue Mühlen entstanden sind und somit Steine entfernt werden müssen. Ist dies der Fall, tritt die anfangs beschriebene Handlung ein. Ist das nicht der Fall und muss auch keine Zieh- oder Sprunghandlung abgeschlossen werden, wechselt der Spieler.

Am Ende wird noch überprüft, ob das Spiel eventuell schon beendet ist. Wenn ja, wird eine Hinweismeldung ausgegeben und die Variable `game_over` auf True gesetzt.

In [ ]:
def make_move(ring, cell):
    global game
    phase = get_player_phase(game.state, game.current_player)
    possible_positions = next_positions(game.state, game.current_player, ring, cell)
    
    if game.number_stones_to_remove == 0:
        if phase == 1:
            if (ring, cell) in possible_positions: # Das geklickte Feld muss frei sein
                game.state[1][ring][cell] = game.current_player # Das geklickte Feld wird zum eigenen Feld
                game.state[0][game.current_player-1] = game.state[0][game.current_player-1]-1 # Die eigenen Steine in remaining um 1 reduzieren
                update_board(game)
            else:
                raise InvalidMoveException()
                print('Du kannst Deinen Stein nur auf ein leeres Feld setzen. Bitte probiere es erneut!')
        elif phase == 2 or phase == 3:
            if game.move_or_jump == False: # -> Stein zum ziehen auswählen
                if game.selected_dot_new in player_pieces(game.state, game.current_player): # prüfen,ob der ausgewählte Stein dem Spieler gehört
                    highlight_positions(possible_positions) # erlaubte Zielfelder anzeigen
                    game.move_or_jump = True
                else:
                    raise InvalidMoveException()
                    print('Das ist nicht Dein Stein. Bitte probiere es erneut!')

            elif game.move_or_jump == True: # -> Zielfeld für Stein wählen
                old_ring, old_cell = game.selected_dot_old
                new_ring, new_cell = game.selected_dot_new
                possible_positions = next_positions(game.state, game.current_player, old_ring, old_cell) # ausgehend vom ausgewählten Stein, erlaubte Zielfelder berechnen
                if (new_ring, new_cell) in possible_positions: # prüfen ob das ausgewählte Feld ein erlaubtes Zielfeld ist
                    game.state[1][new_ring][new_cell] = game.current_player # das Zielfeld als das eigene markieren
                    game.state[1][old_ring][old_cell] = 0 # das ursprüngliche Feld ist jetzt leer
                    game.move_or_jump = False
                    update_board(game)
                else:
                    raise InvalidMoveException()
                    print('Du kannst Deinen Stein nicht auf dieses Feld setzen. Bitte probiere es erneut!')
        else:
            raise InvalidMoveException()
            print('Du befindest Dich anscheinend in keiner gültigen Spielphase. Irgendwas ist schief gegangen... Sorry!')
        
        game.number_stones_to_remove = handle_mills(game)
        if game.number_stones_to_remove > 0:
            highlight_positions(get_opponent_beatable_stones(game.current_player), colour = COLOUR_OPPONENT)
    else:
        selected_ring, selected_cell = game.selected_dot_new
        beatable_stones = get_opponent_beatable_stones(game.current_player)
        if (selected_ring, selected_cell) in beatable_stones:
            game.state[1][selected_ring][selected_cell] = 0
            game.number_stones_to_remove = game.number_stones_to_remove - 1
            update_board(game)
        else:
            raise InvalidMoveException()
            print('Du kannst diesen Stein nicht entfernen. Bitte probiere es erneut!')
        if game.number_stones_to_remove > 0:
            highlight_positions(get_opponent_beatable_stones(game.current_player), colour = COLOUR_OPPONENT)
    
    if game.move_or_jump == False and game.number_stones_to_remove == 0:
        game.current_player = opponent(game.current_player) # Spielerwechsel, wenn nicht noch eine Aktion ausgeführt werden muss
        
    if finish(game.state) != 0: # Prüfen, ob das Spiel zu Ende ist
        game.game_over = True

## Mühlen

Die Funktion `handle_mills()` gibt die Anzahl der im letzten Zug erzeugten Mühlen für den aktuellen Spieler zurück. Hierzu darf der Spieler erst nach dem Ausführen dieser Funktion wechseln.

Im ersten Abschnitt "find all current mills on the board" werden zunächst alle Mühlen identifiziert, die auf dem Spielfeld zu finden sind. Hierzu werden in der ersten Schleife alle Mühlen ermittelt, die sich auf den Ringen befinden. Die zweite Schleife erkennt alle Mühlen, die sich vertikal über alle Ringe erstrecken. Mühlen werden dabei als Tupel der Form ***(Spieler, Ring, Zelle)*** gespeichert. Die Zelle ist in diesem Fall die Zelle einer Mühle mit dem niedrigsten Index. Ein Unterschied besteht bei diesem Format bei den Mühlen, die vertikal verlaufen. Für diese wird der äußerste Ring mit dem Index 0 eingetragen.

Da die Mühlen als Menge gespeichert werden ist es einfach möglich zu bestimmen, ob neue Mühlen entstanden sind. Wurden neue Mühlen erkannt werden durch die Differenz der bestehenden Mühlen zu den existierenden Mühlen die Anzahl der neuen Mühlen bestimmt. Diese Anzahl wird zurückgegeben und die Menge der Mühlen in der Game-Klassen-Instanz aktualisiert.


In [ ]:
def handle_mills(game):
    state = game.state
    player = game.current_player
    old_mills = game.mills
    new_mills = set()
    match = set()
    currentPlayerMills = set()

    #find all current mills on the board
    for ring in range (0,3):
        for cell in [0,2,4,6]:
                if(state[1][ring][cell] == state[1][ring][cell+1] == state[1][ring][(cell+2) % 8] != 0):
                    new_mills.add((state[1][ring][cell], ring ,cell))

    for cell in [1,3,5,7]:
        if(state[1][0][cell] == state[1][1][cell] == state[1][2][cell] != 0):
            new_mills.add((state[1][0][cell], 0 ,cell))

    #set new mills
    game.mills = new_mills   
            

    #if the current mills are the same mills as the old ones or there are now less mills on the board => no new mills
    if(old_mills == new_mills or len(old_mills) > len(new_mills)):
        return(0)


    #find new mills for current player
    # if(old_mills != new_mills):
    #     for mill in new_mills:
    #         for old_mill in old_mills:
    #             if mill == old_mill:
    #                 match.add(mill) 
    # new_mills = new_mills.difference(match) #new_mills are now really the new Mills

    new_mills = new_mills.difference(old_mills) #new_mills are now really the new Mills

    #pics mills that belong to the current player 
    for (mill_owner, ring, cell) in new_mills:
        if mill_owner == player:
            currentPlayerMills.add((mill_owner, ring, cell))

    return len(currentPlayerMills) #returns count of new mills for the current player

`get_opponent_beatable_stones()` gibt eine Liste mit entfernbaren Steinen des Gegners zurück. Dies ist notwendig, wenn auf Grund des Bildens einer Mühle Steine des Gegners entfernt werden dürfen. Laut den Regeln dürfen keine Steine aus Mühlen entfernt werden, es sei denn, alle Steine des Gegners sind in Mühlen. 
Dazu werden alle Mühlsteine des Gegners in einer Menge gespeichert. Anschließend wird geprüft, welche Steine nicht in einer Mühle sind und diese in einer Liste zurückgegeben. Ist die Liste jedoch leer, werden alle Steine des Gegners zurückgegeben.

In [ ]:
def get_opponent_beatable_stones(current_player):
    opponent_player = opponent(current_player)
    opponent_mill_stones = set()

    #find all opponent_mill_stones | wenn ein Stein in einer Mühle ist, darf er nicht entfernt werden
    _,board = game.state

    for ring in range (0,3):
        for cell in [0,2,4,6]:
                if(board[ring][cell] == board[ring][cell+1] == board[ring][(cell+2) % 8] == opponent_player):
                    opponent_mill_stones.add((ring,cell))
                    opponent_mill_stones.add((ring,cell+1))
                    opponent_mill_stones.add((ring,(cell+2) % 8))
    for cell in [1,3,5,7]:
        if(board[0][cell] == board[1][cell] == board[2][cell] == opponent_player):
            opponent_mill_stones.add((0,cell))
            opponent_mill_stones.add((1,cell))
            opponent_mill_stones.add((2,cell))
    
    opponent_stones = player_pieces(game.state, opponent_player)
    beatable_stones = [piece for piece in opponent_stones if piece not in opponent_mill_stones ]
    
    if len(beatable_stones) == 0: #falls kein anderer Stein geschlagen werden kann, dürfen auch Steine in Mühlen geschlagen werden
        return list(opponent_stones)
    return beatable_stones

## Fehler

Die Exception `InvalidMoveException`, wird später in der Funktion `make_move` geworfen, wenn ein ungültiger Spielzug gefordert wird. Dies dient der Fehlerbehandlung.

In [ ]:
class InvalidMoveException(Exception):
    pass